In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from scipy import optimize, stats                     
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error


df_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
df_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_test.head()


In [ ]:
df_test.describe()

In [ ]:
test_id = df_test['ID']

In [ ]:
# first 5 rows of shop name and id
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
shops.head()

Items 

In [ ]:
items_name = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
items_name.head()

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize= (20,20))
words= "".join(items_name['item_name'])
final = WordCloud(width = 2000, height = 800, background_color ="black",min_font_size = 10).generate(words)
plt.imshow(final)
plt.show()

In [ ]:
# first 5 rows of item catory name and id
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items.head()

checking missing vaues in train data

In [ ]:
# checking for missing values
df_train.isnull().sum()

Item category with count

In [ ]:
plt.figure(figsize=(19,9))
sns.barplot(items['item_category_id'], items['item_category_id'])
plt.title('Count for Different Items Categories', fontsize = 30)
plt.xlabel('Item Categories', fontsize = 15)
plt.ylabel('Items in each Categories', fontsize = 15)
plt.show()


In [ ]:
#date is object so we will change Dtype from object to datetime64
df_train['date']=pd.to_datetime(df_train['date'])

In [ ]:
#now we check if there is any null values present in dataset
df_train.isnull().sum()

In [ ]:
df_train['date']=df_train['date'].dt.strftime('%Y-%m')

Drop not required cloumns

In [ ]:
df_train.drop(['date_block_num','item_price'] , axis =1, inplace= True)

In [ ]:
df_train.head().sort_values(by='date')

In [ ]:
df=df_train.groupby(['date','shop_id','item_id']).sum()
df.head()

In [ ]:
df = df.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head()

In [ ]:
df_test= pd.merge(df_test , df , on = ['shop_id', 'item_id'], how = 'left')
df_test.drop(['ID', '2013-01'], axis =1, inplace=True)
df_test= df_test.fillna(0)

In [ ]:
df_test.head()

Model creation

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
from sklearn.pipeline import Pipeline

In [ ]:
df_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(df_submission.shape)
df_submission.head()

In [ ]:
Y_train = df['2015-10'].values
X_train = df.drop(['2015-10'], axis = 1)
X_test = df_test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=101)

In [ ]:
LR = LinearRegression()
LR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, LR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, LR.predict(x_test)))
print('Test set score:', LR.score(x_train,y_train))

In [ ]:
RFR = RandomForestRegressor(n_estimators = 100)
RFR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, RFR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, RFR.predict(x_test)))
print('Test set score:', RFR.score(x_train,y_train))

In [ ]:
prediction = RFR.predict(X_test)

In [ ]:
prediction = list(map(round, prediction))

In [ ]:
prediction = RFR.predict(X_test)

In [ ]:
df_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(df_submission.shape)
df_submission.head()

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_id
sub['item_cnt_month'] = prediction
sub.to_csv('sales.csv',index=False)
print(sub)